In [1]:
from nipype import Node, Workflow, IdentityInterface, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
import numpy as np
import os

In [2]:
data = "/home/paradeisios/Downloads/flanker"
output = "/home/paradeisios/Desktop/nipype_output/preprocessing"

num_of_subjects = 2
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

mni_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm.nii.gz"
mni_brain_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

TR = 2.5

In [3]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [4]:
templates = {"anat": "sub-{subject_list}/anat/sub-{subject_list}_T1w.nii.gz",
             "func": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_bold.nii.gz",
             "conditions": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_events.tsv"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data


substitutions = [("_mean_img","mean_img"),
                 ("_subject_list",""),
                 ("_flirt_func","flirt_func"),
                 ("_func_warp","func_warp")]

substitutions += [("_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_art{:d}".format(i), "art_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("mean_img{:d}".format(i), "mean_img_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_realign{:d}".format(i), "realign_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("flirt_func{:d}".format(i), "flirt_func_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("func_warp{:d}".format(i), "func_warp_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_smooth{:d}".format(i), "smooth_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_vol_removal{:d}".format(i), "vol_removal_{:d}".format(i+1)) for i in range(num_of_runs)]

substitutions += [("seg_0", "GM")]
substitutions += [("seg_1", "WM")]
substitutions += [("seg_2", "CSF")]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [5]:
# structural workflow 

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain


segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural and
    pve0 = grey matter
    pve1 = white matter
    pve2 = CSF """
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true

flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = mni_brain_template
flirt_struct.inputs.dof = 12 #use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"


fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = mni_template
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep


In [6]:
# functional workflow

img_to_float = MapNode(fsl.ImageMaths(),name="img_to_float",iterfield=['in_file'])
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''


vol_removal = MapNode(fsl.ExtractROI(), name="vol_removal",iterfield=['in_file'])
"""This step removes the 5 first volumes to avoid T1 saturation"""
vol_removal.inputs.t_min = 0 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end


realign = MapNode(fsl.MCFLIRT(),name = "realign",iterfield=['in_file'])
"""This step realigns data to the first image"""
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction


mean_img = MapNode(fsl.maths.MeanImage(),name= "mean_img",iterfield=['in_file'])
mean_img.inputs.dimension = "T" # find mean image across the time dimension


flirt_func = MapNode(fsl.FLIRT(),name = "flirt_func",iterfield=['in_file'])
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"


func_warp = MapNode(fsl.ApplyWarp(),name = "func_warp",
                    iterfield=['in_file',"premat"])
"""This step performs non linear registration to standard space using the affine matrix extracted
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = mni_template


smooth = MapNode(fsl.IsotropicSmooth(),name="smooth",iterfield=['in_file'])
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel

In [7]:
preprocessing = Workflow(name="preprocessing")

preprocessing.connect(infosource,"subject_list",selectfiles, "subject_list")
preprocessing.connect(infosource,"subject_list",datasink, "container")                  
preprocessing.connect(selectfiles,"anat",bet, "in_file")  
preprocessing.connect(selectfiles,"anat",fnirt_struct, "in_file") 
                 
preprocessing.connect(bet,"out_file",segment,"in_files")
preprocessing.connect(segment,"restored_image",flirt_struct,"in_file")
preprocessing.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

preprocessing.connect(selectfiles,"func",img_to_float, "in_file")   
preprocessing.connect(img_to_float,"out_file",vol_removal,"in_file")
preprocessing.connect(vol_removal, "roi_file", realign, "in_file")
preprocessing.connect(realign, "out_file", mean_img, "in_file")
preprocessing.connect(mean_img,"out_file",flirt_func,"in_file")
preprocessing.connect(realign,"out_file",func_warp,"in_file")
preprocessing.connect(flirt_func,"out_matrix_file",func_warp,"premat")
preprocessing.connect(segment,"restored_image",flirt_func,"reference")
preprocessing.connect(fnirt_struct,"fieldcoeff_file",func_warp,"field_file")
preprocessing.connect(func_warp,"out_file",smooth,"in_file")

In [8]:
preprocessing.write_graph(graph2use='flat',format='png', simple_form=True,
                          dotfilename= os.path.join(output,"preprocessing_workflow.dot"))

210806-23:18:17,214 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/nipype_output/preprocessing/preprocessing_workflow.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Desktop/nipype_output/preprocessing/preprocessing_workflow.png'

In [9]:
preprocessing.connect(selectfiles,"conditions",datasink,"conditions")
preprocessing.connect(bet,"out_file",datasink,"bet.@bet_brain")
preprocessing.connect(bet,"mask_file",datasink,"bet.@bet_mask")
preprocessing.connect(segment,"restored_image",datasink,"segment.@restored_image")
preprocessing.connect(segment,"tissue_class_files",datasink,"segment.@tissue_class_files")
preprocessing.connect(segment,"bias_field",datasink,"segment.@bias_field")
preprocessing.connect(segment,"tissue_class_map",datasink,"segment.@tissue_class_map")
preprocessing.connect(flirt_struct,"out_file",datasink,"flirt_struct.@structural_flirt")
preprocessing.connect(flirt_struct,"out_matrix_file",datasink,"flirt_struct.@affine_parameters")
preprocessing.connect(fnirt_struct,"warped_file",datasink,"flirt_struct.@structural_fnirt")
preprocessing.connect(fnirt_struct,"field_file",datasink,"flirt_struct.@fieldcoeff_file")
preprocessing.connect(vol_removal,"roi_file",datasink,"vol_removal.@files")
preprocessing.connect(realign,"out_file",datasink,"realign.@realigned_files")
preprocessing.connect(realign,"par_file",datasink,"realign.@realig_parameters")

preprocessing.connect(mean_img,"out_file",datasink,"mean_img.@mean_img")
preprocessing.connect(flirt_func,"out_matrix_file",datasink,"flirt_func.@affine_parameters")
preprocessing.connect(flirt_func,"out_file",datasink,"flirt_func.@functional_flirt")
preprocessing.connect(func_warp,"out_file",datasink,"func_warp.@functional_fnirt")
preprocessing.connect(smooth,"out_file",datasink,"smooth.@smoothed_files")

In [10]:
preprocessing.run(plugin='MultiProc', plugin_args={'n_procs' : 4})

210806-23:18:17,311 nipype.workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
210806-23:18:17,375 nipype.workflow INFO:
	 Running in parallel.
210806-23:18:17,378 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210806-23:18:17,458 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/tmp/tmp9k4p2_hu/preprocessing/_subject_list_02/selectfiles".
210806-23:18:17,458 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/tmp/tmp9lr4eh_q/preprocessing/_subject_list_01/selectfiles".
210806-23:18:17,464 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210806-23:18:17,466 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210806-23:18:17,478 nipype.workflow INFO:
	 [Node] Finished "preprocessing.selectfiles".
210806-23:18:17,486 nipype.workflow INFO:
	 [No

210806-23:18:33,523 nipype.workflow INFO:
	 [Node] Setting-up "_img_to_float1" in "/tmp/tmpv1czu9gf/preprocessing/_subject_list_01/img_to_float/mapflow/_img_to_float1".
210806-23:18:33,541 nipype.workflow INFO:
	 [Node] Running "_img_to_float1" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /home/paradeisios/Downloads/flanker/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz  /tmp/tmpv1czu9gf/preprocessing/_subject_list_01/img_to_float/mapflow/_img_to_float1/sub-01_task-flanker_run-2_bold_maths.nii.gz -odt float
210806-23:18:35,396 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 2 jobs ready. Free memory (GB): 5.94/6.74, Free processors: 0/4.
                     Currently running:
                       * _img_to_float1
                       * _img_to_float0
                       * preprocessing.segment
                       * preprocessing.segment
210806-23:18:40,470 nipype.workflow INFO:
	 [Node] Finished "_img_to_float0".
2108

210806-23:18:57,545 nipype.workflow INFO:
	 [Node] Running "_vol_removal1" ("nipype.interfaces.fsl.utils.ExtractROI"), a CommandLine Interface with command:
fslroi /tmp/tmpv1czu9gf/preprocessing/_subject_list_01/img_to_float/mapflow/_img_to_float1/sub-01_task-flanker_run-2_bold_maths.nii.gz /tmp/tmpk94guzpa/preprocessing/_subject_list_01/vol_removal/mapflow/_vol_removal1/sub-01_task-flanker_run-2_bold_maths_roi.nii.gz 0 -1
210806-23:18:59,421 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 2 jobs ready. Free memory (GB): 5.94/6.74, Free processors: 0/4.
                     Currently running:
                       * _vol_removal1
                       * _vol_removal0
                       * preprocessing.segment
                       * preprocessing.segment
210806-23:19:00,876 nipype.workflow INFO:
	 [Node] Finished "_vol_removal0".
210806-23:19:01,422 nipype.workflow INFO:
	 [Job 32] Completed (_vol_removal0).
210806-23:19:01,443 nipype.workflow INFO:
	 [MultiProc] Runnin

210806-23:20:09,617 nipype.workflow INFO:
	 [Node] Running "_realign1" ("nipype.interfaces.fsl.preprocess.MCFLIRT"), a CommandLine Interface with command:
mcflirt -in /tmp/tmpk94guzpa/preprocessing/_subject_list_01/vol_removal/mapflow/_vol_removal1/sub-01_task-flanker_run-2_bold_maths_roi.nii.gz -dof 6 -out /tmp/tmp9ol3dzve/preprocessing/_subject_list_01/realign/mapflow/_realign1/sub-01_task-flanker_run-2_bold_maths_roi_mcf.nii.gz -mats -plots
210806-23:20:11,491 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 2 jobs ready. Free memory (GB): 5.94/6.74, Free processors: 0/4.
                     Currently running:
                       * _realign1
                       * _realign0
                       * preprocessing.segment
                       * preprocessing.segment
210806-23:20:56,233 nipype.workflow INFO:
	 [Node] Finished "_realign0".
210806-23:20:57,536 nipype.workflow INFO:
	 [Job 36] Completed (_realign0).
210806-23:20:57,561 nipype.workflow INFO:
	 [MultiProc] R

210806-23:21:15,554 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 0 jobs ready. Free memory (GB): 5.94/6.74, Free processors: 0/4.
                     Currently running:
                       * _mean_img1
                       * _mean_img0
                       * preprocessing.segment
                       * preprocessing.segment
210806-23:21:16,224 nipype.workflow INFO:
	 [Node] Finished "_mean_img1".
210806-23:21:16,259 nipype.workflow INFO:
	 [Node] Finished "_mean_img0".
210806-23:21:17,556 nipype.workflow INFO:
	 [Job 40] Completed (_mean_img0).
210806-23:21:17,557 nipype.workflow INFO:
	 [Job 41] Completed (_mean_img1).
210806-23:21:17,565 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 1 jobs ready. Free memory (GB): 6.34/6.74, Free processors: 2/4.
                     Currently running:
                       * preprocessing.segment
                       * preprocessing.segment
210806-23:21:17,652 nipype.workflow INFO:
	 [Node] Setting-up "preprocessi

210806-23:25:36,190 nipype.workflow INFO:
	 [Node] Cached "_flirt_func0" - collecting precomputed outputs
210806-23:25:36,217 nipype.workflow INFO:
	 [Node] "_flirt_func0" found cached.
210806-23:25:36,259 nipype.workflow INFO:
	 [Node] Setting-up "_flirt_func1" in "/tmp/tmpwyhs_ybb/preprocessing/_subject_list_02/flirt_func/mapflow/_flirt_func1".
210806-23:25:36,289 nipype.workflow INFO:
	 [Node] Cached "_flirt_func1" - collecting precomputed outputs
210806-23:25:36,327 nipype.workflow INFO:
	 [Node] "_flirt_func1" found cached.
210806-23:25:36,387 nipype.workflow INFO:
	 [Node] Finished "preprocessing.flirt_func".
210806-23:25:37,800 nipype.workflow INFO:
	 [Job 7] Completed (preprocessing.flirt_func).
210806-23:25:37,815 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 1 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * _flirt_func0
                       * preprocessing.flirt_struct
               

210806-23:42:13,44 nipype.workflow INFO:
	 [Node] Setting-up "_func_warp1" in "/tmp/tmptv91cmkh/preprocessing/_subject_list_01/func_warp/mapflow/_func_warp1".
210806-23:42:13,87 nipype.workflow INFO:
	 [Node] Running "_func_warp0" ("nipype.interfaces.fsl.preprocess.ApplyWarp"), a CommandLine Interface with command:
applywarp --in=/tmp/tmp9ol3dzve/preprocessing/_subject_list_01/realign/mapflow/_realign0/sub-01_task-flanker_run-1_bold_maths_roi_mcf.nii.gz --ref=/home/paradeisios/fsl/data/standard/MNI152_T1_2mm.nii.gz --out=/tmp/tmptv91cmkh/preprocessing/_subject_list_01/func_warp/mapflow/_func_warp0/sub-01_task-flanker_run-1_bold_maths_roi_mcf_warp.nii.gz --warp=/tmp/tmpnsavdtp_/preprocessing/_subject_list_01/fnirt_struct/sub-01_T1w_fieldwarp.nii.gz --premat=/tmp/tmpmayyznmn/preprocessing/_subject_list_01/flirt_func/mapflow/_flirt_func0/f2h_affine.mat
210806-23:42:13,97 nipype.workflow INFO:
	 [Node] Running "_func_warp1" ("nipype.interfaces.fsl.preprocess.ApplyWarp"), a CommandLine Inte

210806-23:47:15,439 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/flirt_struct/_subject_list_01/sub-01_T1w_warped.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/flirt_struct/01/sub-01_T1w_warped.nii.gz
210806-23:47:15,456 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/segment/_subject_list_01/sub-01_T1w_brain_restore.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/segment/01/sub-01_T1w_brain_restore.nii.gz
210806-23:47:15,483 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/segment/_subject_list_01/sub-01_T1w_brain_seg_0.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/segment/01/sub-01_T1w_brain_GM.nii.gz
210806-23:47:15,510 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/segment/_subject_list_01/sub-01_T1w_brain_seg_1.nii.gz -> /home/paradeisios/Desktop/nipype_output/prepr

210806-23:48:23,125 nipype.workflow INFO:
	 [Job 9] Completed (preprocessing.fnirt_struct).
210806-23:48:23,130 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210806-23:48:25,127 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210806-23:48:25,181 nipype.workflow INFO:
	 [Node] Setting-up "_func_warp0" in "/tmp/tmp5siught2/preprocessing/_subject_list_02/func_warp/mapflow/_func_warp0".
210806-23:48:25,185 nipype.workflow INFO:
	 [Node] Setting-up "_func_warp1" in "/tmp/tmp5siught2/preprocessing/_subject_list_02/func_warp/mapflow/_func_warp1".
210806-23:48:25,186 nipype.workflow INFO:
	 [Node] Running "_func_warp0" ("nipype.interfaces.fsl.preprocess.ApplyWarp"), a CommandLine Interface with command:
applywarp --in=/tmp/tmpms3kj4pn/preprocessing/_subject_list_02/realign/mapflow/_realign0/sub-02_task-flanker_run-1_bold_maths_roi_mcf.nii.gz --re

210806-23:53:17,590 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_seg_0.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/02/sub-02_T1w_brain_GM.nii.gz
210806-23:53:17,595 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_seg_1.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/02/sub-02_T1w_brain_WM.nii.gz
210806-23:53:17,597 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_seg_2.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/02/sub-02_T1w_brain_CSF.nii.gz
210806-23:53:17,601 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_bias.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/0